In [4]:
from pprint import pprint
import sys
import getpass

if "./tuleap-rest-api-client/build/lib/" not in sys.path:
    sys.path.append("./tuleap-rest-api-client/build/lib/")

from Tuleap.RestClient.Commons import CertificateVerification
from Tuleap.RestClient.Connection import Connection
from Tuleap.RestClient.Artifacts import Artifacts
from Tuleap.RestClient.Trackers import Tracker
from Tuleap.RestClient.Projects import Projects

from src.ApiClientAddon.CommitAddon import Commits

from src.GitHistory.GitHistory import GitHistory

from src.Graph.ArtifactsWalker import ArtifactsWalker
from src.Graph.ArtifactGraph import ArtifactGraph

In [5]:
keyring.get_keyring()
tuleap_link = "https://tuleap.net/api"  # the link to your tuleap instance (do not forget to add /api)
your_api_key = getpass.getpass()

connection = Connection()
connection.set_access_key(tuleap_link, your_api_key, certificate_verification=CertificateVerification.Enabled)

print(f"Can connect to the API: { 'yes' if connection.call_get_method('/version') else 'no'}")

········
Can connect to the API: yes


You can either build the graph from a project:
(this can be long, it takes around 2 hours for tuleap.net)

In [ ]:
graph = ArtifactGraph()
graph.fill_graph_from_project("Tuleap",
                              ArtifactsWalker(Commits(connection, repository_id=58), Artifacts(connection)),
                              Projects(connection),
                              Tracker(connection))

Or from an artifact or git reference:

In [ ]:
graph = ArtifactGraph()
graph.fill_graph_from_artifact("8e6896fa659ad7bea33a042fe3d5e0c395c10dc8",
                               ArtifactsWalker(Commits(connection, repository_id=58), Artifacts(connection)),
                               through_reverse_links=True,
                               depth_limit=3)
# for an artifact, just give the id
graph.plot_graph()

You can save it to a file:

In [ ]:
graph.save_graph("tuleap_net.graphml")

And re-open it later:

In [ ]:
graph = ArtifactGraph()
graph.load_graph("tuleap_net.graphml")

Here are some functions you can use:

In [ ]:
if graph.check_artifacts_connection("story #26808", "rel #27178", mode="all", types=["None", "_is_child"]):
    print(graph.get_paths())
print()
if graph.search_all_paths_between_artifacts("epic #26754", "story #26808", mode="all", cutoff=10):
    pprint(graph.get_paths(3))



In [ ]:
if graph.request_non_connected_vertices():
    print(graph.get_artifacts())

In [ ]:
# works also with git prefix
graph.search_all_accessible_by_type("story #26808", "rel")
print(graph.get_artifacts())

In [ ]:
git = GitHistory("/path/to/tuleap/")  # local git repository root path

if git.request_overwritten_commits("8e6896fa659ad7bea33a042fe3d5e0c395c10dc8"):
    pprint(git.get_commits())
if git.request_linked_artifacts():
    pprint(git.get_references())